# AG News Classification Dataset

### DESCRIPTION
The AG's news topic classification dataset is constructed by choosing 4 largest classes from the original corpus. Each class contains 30,000 training samples and 1,900 testing samples. The total number of training samples is 120,000 and testing 7,600.

The file classes.txt contains a list of classes corresponding to each label.

The files train.csv and test.csv contain all the training samples as comma-sparated values. There are 3 columns in them, corresponding to class index (1 to 4), title and description. The title and description are escaped using double quotes ("), and any internal double quote is escaped by 2 double quotes (""). New lines are escaped by a backslash followed with an "n" character, that is "\n".

In [1]:
!pip install unidecode contractions

In [2]:
!pip install spacy download en_core_web_sm

In [3]:
!pip install xgboost

In [4]:
!python -m spacy download en_core_web_lg

     -------------------------------------- 400.7/400.7 MB 2.9 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


2022-07-15 18:48:39.373811: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-07-15 18:48:39.375122: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Importing Libraries

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
warnings.filterwarnings('ignore')
import re
import string
import unidecode
import contractions
from sklearn.preprocessing import Normalizer
from nltk.stem.porter import PorterStemmer
from time import time
import spacy
import pprint
from copy import deepcopy
from tqdm import tqdm
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score , f1_score

In C:\Users\sukrav\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\sukrav\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\sukrav\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\sukrav\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\sukrav\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_tes

In [6]:
!pip install -U spacy

  Using cached spacy-3.4.0-cp38-cp38-win_amd64.whl (12.1 MB)
  Using cached thinc-8.1.0-cp38-cp38-win_amd64.whl (1.3 MB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.15
    Uninstalling thinc-8.0.15:
      Successfully uninstalled thinc-8.0.15


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\sukrav\\Anaconda3\\Lib\\site-packages\\~hinc\\backends\\linalg.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [7]:
import spacy
from nltk.stem import PorterStemmer

In [8]:
!pip install xgboost

In [9]:
nlp = spacy.load('en_core_web_lg')
p_stemmer = PorterStemmer()

## Reading the dataset

In [10]:
import os
os.chdir(r"C:\Users\sukrav\Downloads")

In [11]:
df_train=pd.read_csv("ghj.csv")

In [12]:
df_train.shape

(120000, 3)

In [13]:
df_train = df_train.sample(n=30000, random_state =1)

In [14]:
df_train.head()

,Class Index,Title,Description
76467,4,BEA buzzing about Beehive,"BURLINGAME, CALIF.-- BEA Systems is expanding ..."
27991,3,Saudi Violence Pushes Oil Prices Higher,Oil prices were forced higher today by an outb...
62646,1,Storm shows Haiti #39;s straits,Muddy waters still occupied the heart of Gonai...
28374,4,Crews Fail to Find Tree-Killing Beetle (AP),AP - Survey crews searching through woods and ...
98305,2,Justice Is Too Light,It's too bad Stern stopped at 30 games for Ron...


In [15]:
df_train.reset_index(inplace = True, drop = True)

In [16]:
df_train.shape

(30000, 3)

In [17]:
df_train.head()

,Class Index,Title,Description
0,4,BEA buzzing about Beehive,"BURLINGAME, CALIF.-- BEA Systems is expanding ..."
1,3,Saudi Violence Pushes Oil Prices Higher,Oil prices were forced higher today by an outb...
2,1,Storm shows Haiti #39;s straits,Muddy waters still occupied the heart of Gonai...
3,4,Crews Fail to Find Tree-Killing Beetle (AP),AP - Survey crews searching through woods and ...
4,2,Justice Is Too Light,It's too bad Stern stopped at 30 games for Ron...


## Checking for null values

In [18]:
df_train.isnull().sum()

Class Index    0
Title          0
Description    0
dtype: int64

#### No null values found

In [19]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Class Index  30000 non-null  int64 
 1   Title        30000 non-null  object
 2   Description  30000 non-null  object
dtypes: int64(1), object(2)
memory usage: 703.2+ KB


In [20]:
df_train['news'] = df_train['Title'] + ' '+  df_train['Description']

In [21]:
df_train['news'][0]

'BEA buzzing about Beehive BURLINGAME, CALIF.-- BEA Systems is expanding the open source Beehive initiative but still has no plans to participate in the Eclipse open source tools organization, despite the embrace of Beehive by Eclipse.'

In [22]:
df_train['Description'][0]

'BURLINGAME, CALIF.-- BEA Systems is expanding the open source Beehive initiative but still has no plans to participate in the Eclipse open source tools organization, despite the embrace of Beehive by Eclipse.'

In [23]:
df_train.head()

,Class Index,Title,Description,news
0,4,BEA buzzing about Beehive,"BURLINGAME, CALIF.-- BEA Systems is expanding ...","BEA buzzing about Beehive BURLINGAME, CALIF.--..."
1,3,Saudi Violence Pushes Oil Prices Higher,Oil prices were forced higher today by an outb...,Saudi Violence Pushes Oil Prices Higher Oil pr...
2,1,Storm shows Haiti #39;s straits,Muddy waters still occupied the heart of Gonai...,Storm shows Haiti #39;s straits Muddy waters s...
3,4,Crews Fail to Find Tree-Killing Beetle (AP),AP - Survey crews searching through woods and ...,Crews Fail to Find Tree-Killing Beetle (AP) AP...
4,2,Justice Is Too Light,It's too bad Stern stopped at 30 games for Ron...,Justice Is Too Light It's too bad Stern stoppe...


In [24]:
df_train['Class Index'].value_counts()

2    7568
1    7509
4    7471
3    7452
Name: Class Index, dtype: int64

## EDA

In [25]:
# sns.countplot(df_train['Class Index'])

In [26]:
df_train['news'][6]

"O's Lose 10th Straight The A's use home runs by Erubiel Durazo and Eric Chavez to finish a four-game sweep of the crumbling Orioles, 9-4, on Thursday."

In [27]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sukrav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [29]:
# words=[]
# for message in df_train['news']:
#         for word in message.split():
#             if word not in stop_words:
#                 words.append(word)

In [30]:
# from wordcloud import WordCloud
# wc = WordCloud(width=500,height=500,min_font_size=10,background_color='black').generate(str(words).strip())
# plt.figure(figsize=(26,10))
# plt.imshow(wc)
# plt.title(' Most Common  Words',fontsize=30)
# plt.axis('off')
# plt.show()

In [31]:
x= df_train['Class Index'].unique()
x

array([4, 3, 1, 2], dtype=int64)

In [32]:
df_train['word_count'] = df_train['news'].apply(lambda x: len(str(x).split()))

In [33]:
# word_avg = df_train.groupby(df_train['Class Index'])['word_count'].mean().plot(kind='bar', figsize=(30,15))
# plt.xlabel('Class Index', fontsize=35)
# plt.ylabel('Count of Words for Class Index', fontsize=35)
# plt.xticks(fontsize=40,rotation=0)
# plt.yticks(fontsize=40)
# plt.title('Average Number of Words per Class Index', fontsize=70)
# plt.show()

In [34]:
word_avg = df_train.groupby(df_train['Class Index'])['word_count'].mean()
word_avg

Class Index
1    38.930617
2    37.922040
3    37.482958
4    37.302905
Name: word_count, dtype: float64

## Data Cleaning

In [35]:
# to remove numbers/digits

alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
df_train['news']=df_train['news'].map(alphanumeric)

In [36]:
df_train['news'][2]

'Storm shows Haiti # ;s straits Muddy waters still occupied the heart of Gonaives last weekend, three weeks after Tropical Storm Jeanne provoked flash floods that killed some  ,  and left   presumed dead.'

### All the digits have been removed from the data

In [37]:
# #Tokenization
# for i in df_train['news']:
#     print(i.split())

In [38]:
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

# Expanding Contractions in the news data
df_train['news']=df_train['news'].apply(lambda x:expand_contractions(x))

In [39]:
# converting to lower case
df_train['cleaned']=df_train['news'].apply(lambda x: x.lower())

In [40]:
df_train['cleaned'][2]

'storm shows haiti # ;s straits muddy waters still occupied the heart of gonaives last weekend, three weeks after tropical storm jeanne provoked flash floods that killed some  ,  and left   presumed dead.'

In [41]:
# remove digits
df_train['cleaned']=df_train['cleaned'].apply(lambda x: re.sub('\w*\d\w*','', x))

In [42]:
# remove punctuations
df_train['cleaned']=df_train['cleaned'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))

In [43]:
df_train['cleaned'][6]

'o is lose   straight the a is use home runs by erubiel durazo and eric chavez to finish a fourgame sweep of the crumbling orioles    on thursday'

In [44]:
# Removing extra spaces
df_train['cleaned']=df_train['cleaned'].apply(lambda x: re.sub(' +','',x))

In [45]:
df_train['cleaned'][6]

'o is lose straight the a is use home runs by erubiel durazo and eric chavez to finish a fourgame sweep of the crumbling orioles on thursday'

In [46]:
for index,text in enumerate(df_train['cleaned'][450:455]):
  print('News %d:\n'%(index+1),text)

News 1:
 yankees still pondering the enigmatic brown yankees pitcher kevin brown was more animated than usual on the eve of the start that could turn his misguided september punch into an almost amusing memory
News 2:
 unrest is stirring among chinese factory workers dongguan china heralded by an unprecedented series of walkouts the first stirrings of unrest have emerged among the millions of youthful migrant workers who supply seemingly inexhaustible cheap labor for the vast expanse of factories in china is booming pearl river delta
News 3:
 update cirrus logic cuts percent of staff cirrus logic inc cruso quote profile research a maker of microchips for consumer electronics and other applications on wednesday said it had cut its work force by 
News 4:
 death demanded for south african mercenary in e guinea coup bid malabo the prosecutor in the case of south african alleged mercenary leader nick du toit who has admitted a limited role in a coup bid in equatorial guinea called for him t

In [47]:
len(df_train['cleaned']) 

30000

In [48]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Class Index  30000 non-null  int64 
 1   Title        30000 non-null  object
 2   Description  30000 non-null  object
 3   news         30000 non-null  object
 4   word_count   30000 non-null  int64 
 5   cleaned      30000 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.4+ MB


In [49]:
 t0 = time()
t= time() - t0

# Lemmatization with stopwords removal
df_train['lemmatized']=tqdm(df_train['cleaned'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)])))
print (f'processing time : {t:2f} sec')

100%|████████████████████████████████████████████████████████████████████████| 30000/30000 [00:00<00:00, 978278.53it/s]

processing time : 0.000000 sec


In [50]:
df_train['lemmatized'][:72]

0     bea buzzing beehive burlingame calif bea syste...
1     saudi violence push oil price high oil price f...
2     storm show haiti s straits muddy water occupy ...
3     crew fail find treekille beetle ap ap survey c...
4     justice light bad stern stop game ron artest i...
                            ...                        
67    fox join bluray group com october pt twentieth...
68    davenport win germany lindsay davenport win to...
69    opec vote lift output aim ease fear opec minis...
70    black widow revenge terror bombing airliner ru...
71    james young score lebron james score point you...
Name: lemmatized, Length: 72, dtype: object

In [51]:
df_train.head()

,Class Index,Title,Description,news,word_count,cleaned,lemmatized
0,4,BEA buzzing about Beehive,"BURLINGAME, CALIF.-- BEA Systems is expanding ...","BEA buzzing about Beehive BURLINGAME, CALIF.--...",36,bea buzzing about beehive burlingame calif bea...,bea buzzing beehive burlingame calif bea syste...
1,3,Saudi Violence Pushes Oil Prices Higher,Oil prices were forced higher today by an outb...,Saudi Violence Pushes Oil Prices Higher Oil pr...,38,saudi violence pushes oil prices higher oil pr...,saudi violence push oil price high oil price f...
2,1,Storm shows Haiti #39;s straits,Muddy waters still occupied the heart of Gonai...,Storm shows Haiti # ;s straits Muddy waters st...,33,storm shows haiti s straits muddy waters still...,storm show haiti s straits muddy water occupy ...
3,4,Crews Fail to Find Tree-Killing Beetle (AP),AP - Survey crews searching through woods and ...,Crews Fail to Find Tree-Killing Beetle (AP) AP...,43,crews fail to find treekilling beetle ap ap su...,crew fail find treekille beetle ap ap survey c...
4,2,Justice Is Too Light,It's too bad Stern stopped at 30 games for Ron...,Justice Is Too Light It is too bad Stern stopp...,32,justice is too light it is too bad stern stopp...,justice light bad stern stop game ron artest i...


In [52]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [96]:
# Creating Document Term Matrix
from sklearn.feature_extraction.text import TfidfVectorizer
TfidfVectorizer=TfidfVectorizer(analyzer='word', max_features=6708, stop_words=stop_words, ngram_range=(1,2))
data=TfidfVectorizer.fit_transform(df_train['lemmatized'])
df_dtm = pd.DataFrame(data.toarray(), columns=TfidfVectorizer.get_feature_names())
df_dtm.index=df_train.index
df_dtm.head(10)

,aaron,abandon,abbas,abbey,abbey national,abc,abduct,ability,able,aboard,...,yukos,yushchenko,zarqawi,zealand,ziff,ziff davis,zimbabwe,zone,zook,zurich
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
df_dtm.shape

(30000, 10000)

In [55]:
# words=[]
# for message in df_train['lemmatized']:
#         for word in message.split():
#             words.append(word)

In [56]:
# from wordcloud import WordCloud
# string_text = ' '.join(words)    
# wc = WordCloud(width=1000, height=800).generate(string_text)
# #wc = WordCloud(width=500,height=500,min_font_size=10,background_color='black').generate(str(words).strip())
# plt.figure(figsize=(26,10))
# plt.imshow(wc)
# plt.title(' Most Common  Words',fontsize=30)
# plt.axis('off')
# plt.show()

In [57]:
df_train['word_count'] = df_train['lemmatized'].apply(lambda x: len(str(x).split()))

In [58]:
df_train.head()

,Class Index,Title,Description,news,word_count,cleaned,lemmatized
0,4,BEA buzzing about Beehive,"BURLINGAME, CALIF.-- BEA Systems is expanding ...","BEA buzzing about Beehive BURLINGAME, CALIF.--...",23,bea buzzing about beehive burlingame calif bea...,bea buzzing beehive burlingame calif bea syste...
1,3,Saudi Violence Pushes Oil Prices Higher,Oil prices were forced higher today by an outb...,Saudi Violence Pushes Oil Prices Higher Oil pr...,26,saudi violence pushes oil prices higher oil pr...,saudi violence push oil price high oil price f...
2,1,Storm shows Haiti #39;s straits,Muddy waters still occupied the heart of Gonai...,Storm shows Haiti # ;s straits Muddy waters st...,22,storm shows haiti s straits muddy waters still...,storm show haiti s straits muddy water occupy ...
3,4,Crews Fail to Find Tree-Killing Beetle (AP),AP - Survey crews searching through woods and ...,Crews Fail to Find Tree-Killing Beetle (AP) AP...,28,crews fail to find treekilling beetle ap ap su...,crew fail find treekille beetle ap ap survey c...
4,2,Justice Is Too Light,It's too bad Stern stopped at 30 games for Ron...,Justice Is Too Light It is too bad Stern stopp...,15,justice is too light it is too bad stern stopp...,justice light bad stern stop game ron artest i...


#  Modelling

In [60]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
# X2= df_dtm
# y2= df_train['Class Index']

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train,X_test,y_train,y_test = train_test_split(X2,y2,test_size=0.15,random_state=2)

In [ ]:
# from sklearn.svm import SVC

In [ ]:
#svc= SVC()

In [ ]:
#svc.fit(X_train, y_train)

In [ ]:
# y_pred= svc.predict(X_test)
# accuracy_score(y_test, y_pred)*100

In [ ]:
#f1_score(y_test, y_pred, average= None)

## PCA

In [61]:
from sklearn.decomposition import PCA
pca= PCA(n_components = 0.96)
pca.fit(df_dtm)
dim= pca.transform(df_dtm)

In [62]:
X= dim
y= df_train['Class Index']

In [63]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15,random_state=2)

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier


LR= LogisticRegression(n_jobs= -1)
# RF_reg = RandomForestClassifier(n_jobs= -1)
#RFC=RandomForestClassifier(n_jobs= -1)
# KNN=KNeighborsClassifier(n_jobs= -1)
# DTC=DecisionTreeClassifier()
# GBC=GradientBoostingClassifier()
# ABC=AdaBoostClassifier(n_jobs= -1)
#XGBC=XGBClassifier(n_jobs= -1)
#clf=LinearSVC(n_jobs= -1)

models = []

#models.append(("Linear Support Vector Classifier",clf))
models.append(("logistic regression",LR))
# models.append(("K Neighbors classifier",KNN))
# models.append(("DECISION TREE CLASSIFIER",DTC))
#models.append(("RANDOM FOREST CLASSIFIER",RFC))
# models.append(("GRADIENT BOOSTING CLASSIFIER",GBC))
# models.append(("ADA BOOST CLASSIFIER",ABC))
#models.append(("XGB CLASSIFIER",XGBC))
print(models)

[('logistic regression', LogisticRegression(n_jobs=-1))]


In [65]:
for name , classifier in models:
    classifier.fit(X_train , y_train)
    Y_pred=classifier.predict(X_test)
    accuracy = accuracy_score(Y_pred,y_test).round(2)
    print(accuracy*100,name)

91.0 logistic regression


In [79]:
classifier

LogisticRegression(n_jobs=-1)

In [70]:
import pickle

In [98]:
pickle.dump(TfidfVectorizer,open('vectorizer.pkl','wb'))

In [73]:
import os
os.chdir(r"C:\Users\sukrav\Downloads\news")

In [80]:
pickle.dump(classifier,open('model.pkl','wb'))

In [81]:
model = pickle.load(open('model.pkl','rb'))

In [91]:
model.predict([X_test[1]])

array([2], dtype=int64)